In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_selection import chi2, f_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, plot_roc_curve,precision_score, roc_auc_score
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN, SMOTE
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
#Load data
df=pd.read_csv('application_data_clean.csv', index_col = 0)

In [3]:
for i in df.columns:
    print(i)

NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE
CNT_CHILDREN
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE
REGION_POPULATION_RELATIVE
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH
OWN_CAR_AGE
CNT_FAM_MEMBERS
REGION_RATING_CLIENT
REGION_RATING_CLIENT_W_CITY
HOUR_APPR_PROCESS_START
EXT_SOURCE_1
EXT_SOURCE_2
EXT_SOURCE_3
OBS_30_CNT_SOCIAL_CIRCLE
DEF_30_CNT_SOCIAL_CIRCLE
OBS_60_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE
DAYS_LAST_PHONE_CHANGE
AMT_REQ_CREDIT_BUREAU_HOUR
AMT_REQ_CREDIT_BUREAU_DAY
AMT_REQ_CREDIT_BUREAU_WEEK
AMT_REQ_CREDIT_BUREAU_MON
AMT_REQ_CREDIT_BUREAU_QRT
AMT_REQ_CREDIT_BUREAU_YEAR
FLAG_MOBIL
FLAG_EMP_PHONE
FLAG_WORK_PHONE
FLAG_CONT_MOBILE
FLAG_PHONE
FLAG_EMAIL
REG_REGION_NOT_LIVE_REGION
REG_REGION_NOT_WORK_REGION
LIVE

In [4]:
df.AMT_INCOME_TOTAL

SK_ID_CURR
100002.0    202500.0
100003.0    270000.0
100004.0     67500.0
100006.0    135000.0
100007.0    121500.0
              ...   
456251.0    157500.0
456252.0     72000.0
456253.0    153000.0
456254.0    171000.0
456255.0    157500.0
Name: AMT_INCOME_TOTAL, Length: 307511, dtype: float64

In [5]:
df.AMT_CREDIT

SK_ID_CURR
100002.0     406597.5
100003.0    1293502.5
100004.0     135000.0
100006.0     312682.5
100007.0     513000.0
              ...    
456251.0     254700.0
456252.0     269550.0
456253.0     677664.0
456254.0     370107.0
456255.0     675000.0
Name: AMT_CREDIT, Length: 307511, dtype: float64

In [6]:
df.AMT_GOODS_PRICE

SK_ID_CURR
100002.0     351000.0
100003.0    1129500.0
100004.0     135000.0
100006.0     297000.0
100007.0     513000.0
              ...    
456251.0     225000.0
456252.0     225000.0
456253.0     585000.0
456254.0     319500.0
456255.0     675000.0
Name: AMT_GOODS_PRICE, Length: 307511, dtype: float64

In [7]:
df['DEBT_TO_INCOME']=df.AMT_CREDIT/df.AMT_INCOME_TOTAL

In [8]:
df[['TARGET','DEBT_TO_INCOME']]

,TARGET,DEBT_TO_INCOME
SK_ID_CURR,,
100002.0,1,2.007889
100003.0,0,4.790750
100004.0,0,2.000000
100006.0,0,2.316167
100007.0,0,4.222222
...,...,...
456251.0,0,1.617143
456252.0,0,3.743750
456253.0,0,4.429176


In [9]:
correlation = np.abs(df.corr().TARGET)

In [10]:
correlation.sort_values(ascending=False).head(25)

TARGET                         1.000000
EXT_SOURCE_2                   0.160303
EXT_SOURCE_3                   0.157397
EXT_SOURCE_1                   0.099152
DAYS_BIRTH                     0.078239
DAYS_EMPLOYED                  0.070075
REGION_RATING_CLIENT_W_CITY    0.060893
REGION_RATING_CLIENT           0.058899
DAYS_LAST_PHONE_CHANGE         0.055218
DAYS_ID_PUBLISH                0.051457
REG_CITY_NOT_WORK_CITY         0.050994
FLAG_EMP_PHONE                 0.045982
REG_CITY_NOT_LIVE_CITY         0.044395
FLAG_DOCUMENT_3                0.044346
FLOORSMAX_AVG                  0.044003
FLOORSMAX_MEDI                 0.043768
FLOORSMAX_MODE                 0.043226
DAYS_REGISTRATION              0.041975
AMT_GOODS_PRICE                0.039628
REGION_POPULATION_RELATIVE     0.037227
ELEVATORS_AVG                  0.034199
ELEVATORS_MEDI                 0.033863
FLOORSMIN_AVG                  0.033614
FLOORSMIN_MEDI                 0.033394
LIVINGAREA_AVG                 0.032997


In [11]:
correlation.DEBT_TO_INCOME

0.007726797322315164